In [1]:
import psycopg2
from psycopg2 import extras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

%matplotlib inline

In [2]:
def cursor_connect(cursor_factory=None):
    """
    Connects to the DB and returns the connection and cursor, ready to use.
    
    Parameters
    ----------
    cursor_factory : psycopg2.extras
    
    Returns
    -------
    (psycopg2.extensions.connection, psycopg2.extensions.cursor)
        A tuple of (psycopg2 connection, psycopg2 cursor).
    """
    #DB connection
    conn = psycopg2.connect(dbname="mimic", user="mimic", host="localhost", port="2345",
                            password="oNuemmLeix9Yex7W")
    if not cursor_factory:
        cur = conn.cursor()
    else:
        cur = conn.cursor(cursor_factory=cursor_factory)
    return conn, cur

def exec_query(query, curs_dict=False):
    """
    Execute query and returns the SQL output.
    
    Parameters
    ----------
    query: string containing SQL SELECT command
    curs_dict: dict cursor factory (output is dict)
    
    Returns
    -------
    rows: list of rows/records (SQL output)
    """
    if curs_dict == True: 
        conn, cur = cursor_connect(psycopg2.extras.DictCursor)
    else:
        conn, cur = cursor_connect()
    cur.execute(query)
    rows = cur.fetchall()
    return rows

# Dataset

## Interval: 30 Days
Patients' ICU admissions within the last 30 days from the current ICU stay.

## Exclusion Criteria
1. Minors

## Features (`X`)

* **`subjectid`**: subject ID of patients
* **`icustayid`**: ID of a unique ICU stay
* **`readm_days`**: number of days since discharge of previous ICU stay
* **`prev_iculos`**: total length of stay (LOS) of the previous unique ICU stay
* **`N`<sub>icutrav</sub>**: the number of total intra-ICU ward transfers (non-unique ICU stays)
* **`prev_cu`**: categorical feature indicating previous care unit
* **`curr_cu`**: categorical feature indicating current care unit
* **`prev_ICU`**: binary feature indicating if previous care unit was an ICU type
* **`disch_cu`**: categorical feature indicating discharge care unit from ICU stay
* **`disch_ICU`**: binary feature indicating if discharge care unit was an ICU type
* **`icu_in_day`**: time of ICU admission (day/night)
* **`icu_out_day`**: time of ICU discharge (day/night)
* **`intra_risk`**: risk score of intra-period unique ICU readmissions
* Transfer Pairs
    * **`nonicu-MICU`**	
    * **`nonicu-SICU`**	
    * **`nonicu-TSICU`**	
    * **`nonicu-CSRU`**	
    * **`MICU-MICU`**	
    * **`TSICU-TSICU`**	
    * **`nonicu-CCU`**	
    * **`CCU-CCU`**
    * **`CSRU-CSRU`**	
    * **`SICU-SICU`**	
 
## Dependent Variables/Response (`Y`) 
  * **`icu_los`**: ICU length of stay

## Patients (`subject_id`) and their Number of Days till a Unique ICU Readmission

Features:
  * **`subjectid`**: subject ID of patients
  * **`icustaysid`**: ID of a unique ICU stay
  * **`readm_days`**: number of days since discharge of previous ICU stay to admission of current ICU stay (ICU readmission)

In [3]:
# query
q_icustay="""SELECT * FROM
(SELECT subject_id, icustay_id, min_in, max_out,
min_in - lag(max_out)
OVER (PARTITION BY subject_id ORDER BY min_in) AS diff
FROM
(SELECT subject_id, icustay_id, 
MIN(intime) as min_in, MAX(outtime) AS max_out
FROM transfers
WHERE icustay_id IS NOT NULL
GROUP BY subject_id, icustay_id) as sub_q
ORDER BY subject_id) as sub
WHERE diff is not null;
"""

# Query output
icustay = exec_query(q_icustay)
df_icustay_time = pd.DataFrame(icustay, columns=['subjectid', 'icustayid', 
                                            'icu_intime', # first unique ICU admission time 
                                            'icu_outtime', #  unique ICU discharge time
                                            'readm_days']) # number of days since last ICU discharge/transfer
df_icustay_time['icu_prev_outtime'] = df_icustay_time.icu_intime - df_icustay_time.readm_days # time of previous ICU discharge/transfer

# df_icustay['30_prior'] = df_icustay.icu_intime - np.timedelta64(30, 'D')
df_icustay_time.readm_days = np.round(df_icustay_time.readm_days.astype('int') * (1/8.64e13), 3)
df_icustay_time.head()

,subjectid,icustayid,icu_intime,icu_outtime,readm_days,icu_prev_outtime
0,7,236754,2121-05-25 03:26:01,2121-05-25 21:10:19,1.226,2121-05-23 22:01:00
1,17,257980,2135-05-09 14:12:06,2135-05-10 11:18:34,130.839,2134-12-29 18:04:03
2,21,216859,2135-01-30 20:53:34,2135-02-08 05:38:46,135.101,2134-09-17 18:28:32
3,23,234044,2157-10-21 11:40:38,2157-10-22 16:08:48,1507.820,2153-09-04 15:59:11
4,34,290505,2191-02-23 05:25:32,2191-02-24 19:24:10,1679.749,2186-07-19 11:27:20


Extract prior ICUstay ID through `icu_prior_outtime`.

**Note**: Some ICUstays have duplicate entries (use of `DISTINCT ON` prevents duplicates)

In [4]:
q_previcu = """SELECT DISTINCT ON (subject_id, icustay_id, outtime) subject_id, icustay_id, outtime 
FROM transfers 
WHERE icustay_id IS NOT NULL;
"""
# Query output
prev_icustay = exec_query(q_previcu)
df_previcu = pd.DataFrame(prev_icustay, columns=['subjectid', 'prev_icustayid', 'icu_prev_outtime'])

df_icustay = pd.merge(df_icustay_time, df_previcu, on=['subjectid', 'icu_prev_outtime'], how='left')
df_icustay.drop(labels='icu_prev_outtime', axis=1, inplace=True)
df_icustay.head()

,subjectid,icustayid,icu_intime,icu_outtime,readm_days,prev_icustayid
0,7,236754,2121-05-25 03:26:01,2121-05-25 21:10:19,1.226,278444
1,17,257980,2135-05-09 14:12:06,2135-05-10 11:18:34,130.839,277042
2,21,216859,2135-01-30 20:53:34,2135-02-08 05:38:46,135.101,217847
3,23,234044,2157-10-21 11:40:38,2157-10-22 16:08:48,1507.820,227807
4,34,290505,2191-02-23 05:25:32,2191-02-24 19:24:10,1679.749,263086


## Exclusion

#### Neonate Patients

In [5]:
q_nicu = """SELECT DISTINCT icustay_id FROM transfers              
WHERE curr_careunit = 'NICU' AND icustay_id IS NOT NULL;
"""
nicu_stays = exec_query(q_nicu)
df_nicu_stays = pd.DataFrame(nicu_stays, columns=['icustayid'])

df_icustay = df_icustay[df_icustay['icustayid'].isin(df_nicu_stays.icustayid)==False]
df_icustay.shape

(14826, 6)

#### Minors

In [6]:
# age of patients < 90
q_age_hadm1 = """SELECT a.subject_id,
    FLOOR((a.admittime::date - p.dob::date)/365.0) AS age
    FROM admissions as a
    INNER JOIN patients as p
    ON a.subject_id = p.subject_id
    WHERE FLOOR((a.admittime::date - p.dob::date)/365.0) < 90;"""

# adjusted age of patients > 89
q_age_hadm2 = """SELECT a.subject_id,
    FLOOR((a.admittime::date - p.dob::date)/365.0) -210 AS age
    FROM admissions as a
    INNER JOIN patients as p
    ON a.subject_id = p.subject_id
    WHERE FLOOR((a.admittime::date - p.dob::date)/365.0) > 89;"""

age_hadm1 = exec_query(q_age_hadm1, False)
age_hadm2 = exec_query(q_age_hadm2, False)
df_age_hadm1 = pd.DataFrame(age_hadm1, columns=['subjectid', 'age'])
df_age_hadm2 = pd.DataFrame(age_hadm2, columns=['subjectid', 'age'])

df_age_hadm = pd.concat([df_age_hadm1, df_age_hadm2])
df_adults = df_age_hadm[df_age_hadm.age > 17]
df_adults_sid = list(df_adults.subjectid.value_counts().index.sort_values())

df_icustay = df_icustay[df_icustay.subjectid.isin(df_adults_sid)]
df_icustay.shape

(14824, 6)

## Dataset Interval/Period 

### Count for each Cutoff
Cutoff for days between last ICU stay discharge and current ICU admission.

In [7]:
count_disch = dict()
for elem in [30, 60, 90, 120, 150, 180]:
    count_disch[elem] = df_icustay[df_icustay.readm_days <= elem].shape[0]

df_count = pd.DataFrame.from_dict(count_disch, orient='index').reset_index()
df_count.columns = ['days_cutoff', 'count']
df_count.sort_values('days_cutoff', ascending=True)

,days_cutoff,count
5,30,6161
4,60,7626
3,90,8494
2,120,9061
1,150,9525
0,180,9914


### Interval: 30 Days

In [8]:
period = 30
df_icustay_p = df_icustay[df_icustay['readm_days'] <= period]

### Previous ICU LOS
The total LOS of the previous unique ICU stay, *includes the duration of all intra-ICU stays*.

In [9]:
q_prevlos = """SELECT icustay_id, los 
FROM icustays;"""

prevlos = exec_query(q_prevlos)
df_prevlos = pd.DataFrame(prevlos, columns=['prev_icustayid', 'prev_iculos'])

df_icustay1 = pd.merge(df_icustay_p, df_prevlos, on='prev_icustayid', how='left')

print df_icustay1.shape
df_icustay1.head()

(6161, 7)


,subjectid,icustayid,icu_intime,icu_outtime,readm_days,prev_icustayid,prev_iculos
0,36,211200,2131-05-16 23:18:26,2131-05-23 19:56:11,11.425,280987,1.1096
1,41,237024,2133-01-09 12:18:30,2133-01-12 15:51:03,2.845,261027,3.3937
2,68,225771,2173-12-31 01:52:46,2173-12-31 21:33:34,11.418,294232,3.5368
3,68,272667,2174-01-08 13:12:06,2174-01-14 22:45:42,7.652,225771,0.8200
4,91,256972,2177-05-07 03:52:00,2177-05-10 15:16:00,9.576,218528,0.4965


### N<sub>icutrav</sub>
The number of total *ICU* ward transfers for each patient's unique ICU stay. The value indicates the number of (non-unique) intra-ICU ward transfers.

In [10]:
q_multtrav = """SELECT icustay_id, COUNT(*)
FROM transfers
WHERE icustay_id IS NOT NULL
GROUP BY icustay_id"""

mult_trav = exec_query(q_multtrav)
df_multtrav = pd.DataFrame(mult_trav, columns=['icustayid', 'n_icutrav'])
df_icustay2 = pd.merge(df_icustay1, df_multtrav, on='icustayid', how='left')

print df_icustay2.shape
df_icustay2.head()

(6161, 8)


,subjectid,icustayid,icu_intime,icu_outtime,readm_days,prev_icustayid,prev_iculos,n_icutrav
0,36,211200,2131-05-16 23:18:26,2131-05-23 19:56:11,11.425,280987,1.1096,3
1,41,237024,2133-01-09 12:18:30,2133-01-12 15:51:03,2.845,261027,3.3937,1
2,68,225771,2173-12-31 01:52:46,2173-12-31 21:33:34,11.418,294232,3.5368,1
3,68,272667,2174-01-08 13:12:06,2174-01-14 22:45:42,7.652,225771,0.8200,1
4,91,256972,2177-05-07 03:52:00,2177-05-10 15:16:00,9.576,218528,0.4965,1


### Current Care Unit
Type of ICU admitted/transferred into.

**`prev_cu`**: categorical feature indicating previous care unit  
**`curr_cu`**: categorical feature indicating current care unit  
**`prev_ICU`**: binary feature indicating previous ICU  

Legend:
  * nonICU : 0
  * MICU : 1
  * CSRU : 2
  * SICU : 3
  * CCU : 4 
  * TSICU : 5
  * NICU :6 
  * NWARD :7

In [11]:
def binary_cu(careunit):
    if careunit > 0 and careunit < 7:
        x = 1
    else:
        x = 0
    return x

In [12]:
q_careunit = """SELECT DISTINCT ON (icustay_id, intime) icustay_id, intime, curr_careunit, prev_careunit
FROM transfers WHERE icustay_id IS NOT NULL;"""

careunit = exec_query(q_careunit)
df_careunit = pd.DataFrame(careunit, 
                           columns = ['icustayid', 'icu_intime', 'curr_cu', 'prev_cu'])

df_careunit.prev_cu.replace(to_replace = 
                            {'':0, 'MICU':1, 'CSRU': 2, 'SICU': 3, 'CCU': 4, 
                             'TSICU': 5,  'NICU':6, 'NWARD':7}, inplace=True)
df_careunit.curr_cu.replace(to_replace = 
                            {'':0, 'MICU':1, 'CSRU': 2, 'SICU': 3, 'CCU': 4, 
                             'TSICU': 5,  'NICU':6, 'NWARD':7}, inplace=True)

df_careunit['prev_ICU'] = df_careunit.prev_cu.apply(binary_cu)
# df_careunit['curr_ICU'] = df_careunit.curr_cu.apply(binary_cu)

df_icustay3 = pd.merge(df_icustay2, df_careunit, on=['icustayid', 'icu_intime'], how='left')

print df_icustay3.shape
df_icustay3.head()

(6161, 11)


,subjectid,icustayid,icu_intime,icu_outtime,readm_days,prev_icustayid,prev_iculos,n_icutrav,curr_cu,prev_cu,prev_ICU
0,36,211200,2131-05-16 23:18:26,2131-05-23 19:56:11,11.425,280987,1.1096,3,2,0,0
1,41,237024,2133-01-09 12:18:30,2133-01-12 15:51:03,2.845,261027,3.3937,1,2,0,0
2,68,225771,2173-12-31 01:52:46,2173-12-31 21:33:34,11.418,294232,3.5368,1,1,0,0
3,68,272667,2174-01-08 13:12:06,2174-01-14 22:45:42,7.652,225771,0.8200,1,1,0,0
4,91,256972,2177-05-07 03:52:00,2177-05-10 15:16:00,9.576,218528,0.4965,1,1,0,0


### ICU Discharge Care Unit
Ward patient was transferred to after ICU stay.

  * **`disch_cu`**: categorical feature indicating discharge unit from ICU
  * **`disch_ICU`**: binary feature indicating ICU discharge

Legend:
  * nonICU : 0
  * MICU : 1
  * CSRU : 2
  * SICU : 3
  * CCU : 4 
  * TSICU : 5
  * NICU :6 
  * NWARD :7
  
**Note**: Some records are missing information such as transfer from an ICU, indicating the end of the ICU stay. These `NULL` rows have been omitted through an inner join.

In [13]:
q_disch = """SELECT DISTINCT ON (t1.outtime) t1.subject_id, t1.icustay_id, t2.curr_careunit, t1.outtime 
FROM
  (SELECT * FROM transfers WHERE curr_careunit LIKE '%U') as t1
INNER JOIN
  (SELECT * FROM transfers WHERE prev_careunit != '') as t2
ON t1.outtime = t2.intime"""

disch_unit = exec_query(q_disch)
df_disch = pd.DataFrame(disch_unit, 
                        columns=['subjectid', 'icustayid', 'disch_cu', 'icu_outtime'])
df_disch['disch_cu'].replace(to_replace = 
                            {'':0, 'MICU':1, 'CSRU': 2, 'SICU': 3, 'CCU': 4, 
                             'TSICU': 5,  'NICU':6, 'NWARD':7}, inplace=True)

df_disch['disch_ICU'] = df_disch.disch_cu.apply(binary_cu)

df_disch

df_icustay4 = pd.merge(df_icustay3, df_disch[['icustayid', 'disch_cu', 'icu_outtime', 'disch_ICU']], 
                       on=['icustayid', 'icu_outtime'], how='inner')
print df_icustay4.shape
df_icustay4.head()

(6160, 13)


,subjectid,icustayid,icu_intime,icu_outtime,readm_days,prev_icustayid,prev_iculos,n_icutrav,curr_cu,prev_cu,prev_ICU,disch_cu,disch_ICU
0,36,211200,2131-05-16 23:18:26,2131-05-23 19:56:11,11.425,280987,1.1096,3,2,0,0,0,0
1,41,237024,2133-01-09 12:18:30,2133-01-12 15:51:03,2.845,261027,3.3937,1,2,0,0,0,0
2,68,225771,2173-12-31 01:52:46,2173-12-31 21:33:34,11.418,294232,3.5368,1,1,0,0,0,0
3,68,272667,2174-01-08 13:12:06,2174-01-14 22:45:42,7.652,225771,0.8200,1,1,0,0,0,0
4,91,256972,2177-05-07 03:52:00,2177-05-10 15:16:00,9.576,218528,0.4965,1,1,0,0,0,0


### ICU Time (Day/Night)
Time of ICU "admission" and "discharge" (day or night)

Features
  * **`icu_in_day`**: Time of ICU admission
  * **`icu_out_day`**: Time of ICU discharge

Legend:
  * **`0`**: night
  * **`1`**: day

In [14]:
def day_night(datetime):
    hour = np.timedelta64(np.datetime64(datetime, 'h') - (np.datetime64(datetime, 'D')), 'h')
    if hour.astype(np.int64) >=6 and hour.astype(np.int64) <=18:
        time = 1 # day
    else:
        time = 0 # night
    return time

In [15]:
df_icustay4['icu_in_day'] = df_icustay4['icu_intime'].apply(day_night)
df_icustay4['icu_out_day'] = df_icustay4['icu_outtime'].apply(day_night)
print df_icustay4.shape
df_icustay4.head()

(6160, 15)


,subjectid,icustayid,icu_intime,icu_outtime,readm_days,prev_icustayid,prev_iculos,n_icutrav,curr_cu,prev_cu,prev_ICU,disch_cu,disch_ICU,icu_in_day,icu_out_day
0,36,211200,2131-05-16 23:18:26,2131-05-23 19:56:11,11.425,280987,1.1096,3,2,0,0,0,0,0,0
1,41,237024,2133-01-09 12:18:30,2133-01-12 15:51:03,2.845,261027,3.3937,1,2,0,0,0,0,1,1
2,68,225771,2173-12-31 01:52:46,2173-12-31 21:33:34,11.418,294232,3.5368,1,1,0,0,0,0,0,0
3,68,272667,2174-01-08 13:12:06,2174-01-14 22:45:42,7.652,225771,0.8200,1,1,0,0,0,0,1,0
4,91,256972,2177-05-07 03:52:00,2177-05-10 15:16:00,9.576,218528,0.4965,1,1,0,0,0,0,0,1


### ICU Intra-Interval Readmission
Risk Scores based on intra-period ICU readmissions (*multiple unique ICU admissions within the specified interval*).

**`Risk Score = N`<sub>ICU readmission</sub> ` = N`<sub>Total ICU admissions</sub>` - 1`**  
The value of the risk score indicates the count/frequency of readmission for the given ICU stay.
 * `0`: indicates first ICU stay during the interval
 * `2`: indicates 3 total ICU stays and 2 ICU readmissions during the interval 

In [16]:
# df_icustay4['icustayid'].groupby(df_icustay4['subjectid']).agg('count'-9'')
d_risk = dict()
for i, row in df_icustay4.iterrows():
    if d_risk.has_key(row.subjectid):
        d_risk[row.subjectid]['count'] += 1
        d_risk[row.subjectid][row.icustayid] = d_risk[row.subjectid]['count'] 

    else:
        d_icu = {'count': 0}
        d_icu[row.icustayid] = d_icu['count']
        d_risk[row.subjectid] = d_icu
        
        
df_icustay4['intra_risk'] = df_icustay4.subjectid.astype(str)+'-'+df_icustay4.astype(str).icustayid

In [17]:
def intra_interval(stay):
    risk_score = str.split(stay, '-')
    sid = int(risk_score[0])
    stayid = int(risk_score[1])
    
    return d_risk[sid][stayid]    

In [18]:
df_icustay4['intra_risk'] = df_icustay4['intra_risk'].apply(intra_interval)
print df_icustay4.shape
df_icustay4.head()

(6160, 16)


,subjectid,icustayid,icu_intime,icu_outtime,readm_days,prev_icustayid,prev_iculos,n_icutrav,curr_cu,prev_cu,prev_ICU,disch_cu,disch_ICU,icu_in_day,icu_out_day,intra_risk
0,36,211200,2131-05-16 23:18:26,2131-05-23 19:56:11,11.425,280987,1.1096,3,2,0,0,0,0,0,0,0
1,41,237024,2133-01-09 12:18:30,2133-01-12 15:51:03,2.845,261027,3.3937,1,2,0,0,0,0,1,1,0
2,68,225771,2173-12-31 01:52:46,2173-12-31 21:33:34,11.418,294232,3.5368,1,1,0,0,0,0,0,0,0
3,68,272667,2174-01-08 13:12:06,2174-01-14 22:45:42,7.652,225771,0.8200,1,1,0,0,0,0,1,0,1
4,91,256972,2177-05-07 03:52:00,2177-05-10 15:16:00,9.576,218528,0.4965,1,1,0,0,0,0,0,1,0


### Traversal Pairs
**Feature transformation** The probability of the feature combination pair is used as weight and applied onto the count for each row.

**Note**: The overall hospital admission LOS may be extracted by taking the aggregate mean of the DataFrame, after it has been grouped by `subject_id`.

In [19]:
q_trav = """SELECT subject_id, icustay_id, eventtype,
prev_careunit, curr_careunit
FROM transfers
WHERE icustay_id IS NOT NULL;"""
mult_trav = exec_query(q_trav, False)
mult_col = ['subjectid', 'icustayid', 'eventtype', 'prev_cu', 'curr_cu']
df_trav = pd.DataFrame(mult_trav, columns = mult_col)
df_trav.replace(to_replace='', value=np.nan, inplace=True, regex=True)

df_trav = df_trav[df_trav.prev_cu != 'NICU']
df_trav = df_trav[df_trav.prev_cu != 'NWARD']
df_trav = df_trav[df_trav.curr_cu != 'NICU']
df_trav = df_trav[df_trav.curr_cu != 'NWARD']
df_trav.prev_cu.fillna('nonicu', inplace=True)
df_trav.curr_cu.fillna('nonicu', inplace=True)

df_trav['trans'] = df_trav.prev_cu+'-'+df_trav.curr_cu

# Filter for Patients with ICU readmission
q_icupat="""SELECT * FROM
    (SELECT subject_id, COUNT(icustay_id) AS n_icustays
    FROM icustays
    GROUP BY subject_id) AS sub_q
WHERE n_icustays > 1;"""

icupat = exec_query(q_icupat)
df_icupat = pd.DataFrame(icupat, columns=['subjectid', 'n_icustays'])
df_icupat
# filter for ICU patients with readmissions
filter_preadm = list(df_icupat.subjectid)
df_trav = df_trav[df_trav.subjectid.isin(filter_preadm)]

In [20]:
from collections import Counter

icuid = list(df_trav.icustayid.value_counts().index) # unique subject_id

main_d = dict()
for stay in icuid:
    pair_d = dict(Counter(df_trav[df_trav.icustayid==stay].trans))
    pair_d['icustayid'] = stay # add subjectid key
    main_d[stay] = pair_d

#### Count of Traversal Pair

In [21]:
df_toppairs = df_trav.trans.value_counts(ascending=False).to_frame()
df_top = df_toppairs.transpose().iloc[:,0:11]

df_pairct = pd.DataFrame.from_dict(main_d, orient='index')

# drop non-top trans pair cols
pairs_drop = list(df_toppairs.iloc[10:].index)
df_pairct.drop(pairs_drop, axis=1, inplace=True) 

df_icustay5 = pd.merge(df_icustay4, df_pairct, on='icustayid', how='left')

#### Probability Transformation (Weight)

In [22]:
prev_cu = df_trav.prev_cu
curr_cu = df_trav.curr_cu
pair_prob = pd.crosstab(prev_cu, curr_cu) / pd.crosstab(prev_cu, curr_cu).sum()
pair_prob

curr_cu,CCU,CSRU,MICU,SICU,TSICU
prev_cu,,,,,
CCU,0.058462,0.029022,0.018337,0.009021,0.008323
CSRU,0.039126,0.230929,0.008767,0.011963,0.007491
MICU,0.039126,0.012852,0.100891,0.036478,0.032043
SICU,0.012056,0.006633,0.015488,0.083742,0.014149
TSICU,0.009099,0.003731,0.010520,0.018631,0.110695
nonicu,0.842129,0.716833,0.845996,0.840165,0.827299


In [23]:
df_pairprob = pair_prob.unstack().to_frame(name='prob').reset_index()
df_pairprob['trans'] = df_pairprob.prev_cu+'-'+df_pairprob.curr_cu
df_pairprob.drop(['curr_cu', 'prev_cu'], axis=1, inplace=True)
df_pairprob.set_index('trans', drop=True, inplace=True)
# df_pairprob.sort_values('prob', ascending=False).head()

In [24]:
pairs = ['nonicu-MICU', 'nonicu-SICU', 'nonicu-TSICU', 'nonicu-CSRU',
         'MICU-MICU', 'TSICU-TSICU', 'nonicu-CCU', 'CCU-CCU', 'CSRU-CSRU',
         'SICU-SICU']

for elem in pairs:
    df_icustay5[elem].fillna(0, inplace=True)
    df_icustay5[elem] = np.round(df_icustay5[elem] * df_pairprob.loc[elem].values[0],3)
    
print df_icustay5.shape
df_icustay5.head()

(6160, 26)


,subjectid,icustayid,icu_intime,icu_outtime,readm_days,prev_icustayid,prev_iculos,n_icutrav,curr_cu,prev_cu,...,nonicu-SICU,nonicu-CSRU,nonicu-MICU,MICU-MICU,nonicu-TSICU,nonicu-CCU,TSICU-TSICU,SICU-SICU,CSRU-CSRU,CCU-CCU
0,36,211200,2131-05-16 23:18:26,2131-05-23 19:56:11,11.425,280987,1.1096,3,2,0,...,0.0,0.717,0.000,0.0,0.0,0.0,0.0,0.0,0.462,0.0
1,41,237024,2133-01-09 12:18:30,2133-01-12 15:51:03,2.845,261027,3.3937,1,2,0,...,0.0,0.717,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0
2,68,225771,2173-12-31 01:52:46,2173-12-31 21:33:34,11.418,294232,3.5368,1,1,0,...,0.0,0.000,0.846,0.0,0.0,0.0,0.0,0.0,0.000,0.0
3,68,272667,2174-01-08 13:12:06,2174-01-14 22:45:42,7.652,225771,0.8200,1,1,0,...,0.0,0.000,0.846,0.0,0.0,0.0,0.0,0.0,0.000,0.0
4,91,256972,2177-05-07 03:52:00,2177-05-10 15:16:00,9.576,218528,0.4965,1,1,0,...,0.0,0.000,0.846,0.0,0.0,0.0,0.0,0.0,0.000,0.0


## Response Variable
### Average ICU LOS
Feature (Response): The average length of stay in the ICU (`icustay_id`) for each patient's hospital admission.
  * Feature is the response/dependent variable (`y`)

In [25]:
q_iculos = """SELECT icustay_id, los 
FROM icustays;"""

iculos = exec_query(q_iculos)
df_iculos = pd.DataFrame(iculos, columns=['icustayid', 'icu_los'])

df_icustay_f = pd.merge(df_icustay5, df_iculos, on='icustayid', how='left')
df_icustay_f.dropna(inplace=True)

print df_icustay_f.shape
df_icustay_f.head()

(6160, 27)


,subjectid,icustayid,icu_intime,icu_outtime,readm_days,prev_icustayid,prev_iculos,n_icutrav,curr_cu,prev_cu,...,nonicu-CSRU,nonicu-MICU,MICU-MICU,nonicu-TSICU,nonicu-CCU,TSICU-TSICU,SICU-SICU,CSRU-CSRU,CCU-CCU,icu_los
0,36,211200,2131-05-16 23:18:26,2131-05-23 19:56:11,11.425,280987,1.1096,3,2,0,...,0.717,0.000,0.0,0.0,0.0,0.0,0.0,0.462,0.0,6.8595
1,41,237024,2133-01-09 12:18:30,2133-01-12 15:51:03,2.845,261027,3.3937,1,2,0,...,0.717,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,3.1476
2,68,225771,2173-12-31 01:52:46,2173-12-31 21:33:34,11.418,294232,3.5368,1,1,0,...,0.000,0.846,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.8200
3,68,272667,2174-01-08 13:12:06,2174-01-14 22:45:42,7.652,225771,0.8200,1,1,0,...,0.000,0.846,0.0,0.0,0.0,0.0,0.0,0.000,0.0,6.3983
4,91,256972,2177-05-07 03:52:00,2177-05-10 15:16:00,9.576,218528,0.4965,1,1,0,...,0.000,0.846,0.0,0.0,0.0,0.0,0.0,0.000,0.0,3.4750


## Composite Dataset
Drop unneeded features that were used for data engineering/feature extraction.
  * **`icu_intime`**
  * **`icu_outtime`**
  * **`prev_icustayid`**

In [26]:
df_icustay_f.drop(['icu_intime', 'icu_outtime', 'prev_icustayid'], axis=1, inplace=1)

print df_icustay_f.shape
df_icustay_f.head()

(6160, 24)


,subjectid,icustayid,readm_days,prev_iculos,n_icutrav,curr_cu,prev_cu,prev_ICU,disch_cu,disch_ICU,...,nonicu-CSRU,nonicu-MICU,MICU-MICU,nonicu-TSICU,nonicu-CCU,TSICU-TSICU,SICU-SICU,CSRU-CSRU,CCU-CCU,icu_los
0,36,211200,11.425,1.1096,3,2,0,0,0,0,...,0.717,0.000,0.0,0.0,0.0,0.0,0.0,0.462,0.0,6.8595
1,41,237024,2.845,3.3937,1,2,0,0,0,0,...,0.717,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,3.1476
2,68,225771,11.418,3.5368,1,1,0,0,0,0,...,0.000,0.846,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.8200
3,68,272667,7.652,0.8200,1,1,0,0,0,0,...,0.000,0.846,0.0,0.0,0.0,0.0,0.0,0.000,0.0,6.3983
4,91,256972,9.576,0.4965,1,1,0,0,0,0,...,0.000,0.846,0.0,0.0,0.0,0.0,0.0,0.000,0.0,3.4750


In [27]:
df_icustay_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6160 entries, 0 to 6159
Data columns (total 24 columns):
subjectid       6160 non-null int64
icustayid       6160 non-null int64
readm_days      6160 non-null float64
prev_iculos     6160 non-null float64
n_icutrav       6160 non-null int64
curr_cu         6160 non-null int64
prev_cu         6160 non-null int64
prev_ICU        6160 non-null int64
disch_cu        6160 non-null int64
disch_ICU       6160 non-null int64
icu_in_day      6160 non-null int64
icu_out_day     6160 non-null int64
intra_risk      6160 non-null int64
nonicu-SICU     6160 non-null float64
nonicu-CSRU     6160 non-null float64
nonicu-MICU     6160 non-null float64
MICU-MICU       6160 non-null float64
nonicu-TSICU    6160 non-null float64
nonicu-CCU      6160 non-null float64
TSICU-TSICU     6160 non-null float64
SICU-SICU       6160 non-null float64
CSRU-CSRU       6160 non-null float64
CCU-CCU         6160 non-null float64
icu_los         6160 non-null float64
dty

In [28]:
df_icustay_f.describe()

,subjectid,icustayid,readm_days,prev_iculos,n_icutrav,curr_cu,prev_cu,prev_ICU,disch_cu,disch_ICU,...,nonicu-CSRU,nonicu-MICU,MICU-MICU,nonicu-TSICU,nonicu-CCU,TSICU-TSICU,SICU-SICU,CSRU-CSRU,CCU-CCU,icu_los
count,6160.000000,6160.000000,6160.000000,6160.000000,6160.000000,6160.000000,6160.000000,6160.000000,6160.0,6160.0,...,6160.000000,6160.000000,6160.000000,6160.000000,6160.000000,6160.000000,6160.000000,6160.000000,6160.000000,6160.000000
mean,35899.599026,249426.524026,9.127547,5.150684,1.301299,2.263636,0.000325,0.000162,0.0,0.0,...,0.128501,0.387567,0.004591,0.073978,0.116458,0.000936,0.001812,0.013575,0.000556,5.108498
std,28844.497122,28899.427910,7.784120,7.588340,0.661220,1.350546,0.025482,0.012741,0.0,0.0,...,0.294932,0.455978,0.024462,0.251552,0.303490,0.012110,0.013520,0.068698,0.006306,7.725125
min,36.000000,200001.000000,0.001000,0.000100,1.000000,1.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000300
25%,12847.000000,224024.500000,2.811750,1.403475,1.000000,1.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.440675
50%,26229.000000,249333.500000,6.219000,2.674350,1.000000,2.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.741000
75%,58518.250000,274093.750000,13.956750,5.337550,1.000000,3.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.846000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.344900
max,99865.000000,299994.000000,29.997000,116.832700,9.000000,5.000000,2.000000,1.000000,0.0,0.0,...,2.867000,3.384000,0.404000,2.482000,1.684000,0.443000,0.251000,1.155000,0.175000,173.072500


# Predictive Modeling

In [29]:
import sys, os
src_abspath = os.path.abspath(os.path.join(os.path.split(os.getcwd())[0], 'src'))
sys.path.append(src_abspath)
from db import *
from clf import *

/home/cpnguyen/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/cpnguyen/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [30]:
am_debugging = True
if am_debugging:
    np.random.seed(2)
else:
    np.random.seed()

#### Partition: Training & Testing Sets

In [31]:
data = df_icustay_f.copy().iloc[:1000, :]
X_train, X_test, y_train, y_test = data_partition(data)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((700, 23), (300, 23), (700,), (300,))

#### Normalization

In [32]:
scaled_X_train, scaled_X_test = scale(X_train, X_test)

#### Feature Selection

In [33]:
pass

## Model: Linear Regression

In [34]:
clf_model, y_pred, best_p, best_score = gridsearch(scaled_X_train, scaled_X_test, y_train)
print "Best Parameters: ", best_p
print "Best Grid Search Score: ", best_score
print "Best Estimator: ", clf_model, "\n"

Best Parameters:  {'kernel': 'linear', 'C': 1000, 'gamma': 0.0001}
Best Grid Search Score:  0.0510005223002
Best Estimator:  SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.0001,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False) 



#### Compare Predictions

In [35]:
y_pred[-10:]

array([  2.53905234,   2.55147744,   6.33025238,   2.33192863,
         2.53241604,   2.83256744,  13.31739346,   2.91845799,
         1.55092954,   6.40806963])

In [36]:
y_test[-10:]

array([  4.6149,   6.8684,  13.8397,   0.3632,   1.6397,   3.0585,
         3.0521,   0.2826,   3.5426,   0.8828])

### Metrics

#### Root Mean Squared Error (RMSE)
Indicates the *absolute fit* of the model to the data. In other words, how close the observed data points are to the model's predicted values.

Also, it is the standard deviation of unexplained variance (residuals).

In [37]:
rmse = RMSE(y_pred, y_test)
print "RMSE = %.3f days" % rmse

RMSE = 10.684 days


In [38]:
print "[%.2f,%.2f]" % (y_test.min(), y_test.max())

[0.00,71.01]


The linear regression model has an RMSE value of 10.68 days. The value is somewhat small relative to the range of the `dependent variable` (average ICU LOS). Thus, indicating that model has decent accuracy in predicting the response.

#### Mean Absolute Error
The Mean Absolute Error measures how close the model's predictions are to the observed values.

In [39]:
mae = MAE(y_pred, y_test)
print "MAE = %.3f days" % mae

MAE = 1.643 days


The average difference between prediction and observation is 1.64 days, which is low.

#### R<sup>2</sup>: Coefficient of Determination
R<sup>2</sup> quantifies the goodness of fit of the linear model. More specifically, it depicts the predictive power of the model.

Range: [0, 1]

In [40]:
r2(y_pred, y_test)

0.074394243452860609

The R<sup>2</sup> value is approximately 0.074, which is low and indicates that the model does not have strong predictive power.

## Model: Support Vector Regression

In [41]:
def gridsearch(X_train, X_test, y_train):
    """
    Function determines the optimal parameters of the best classifier model/estimator by performing a grid search.
    The best model will be fitted with the Training set and subsequently used to predict the classification/labels
    of the Testing set. The function returns the "best" classifier instance, classifier predictions, best parameters,
    and grid score.

    :param X_train: Training set features
    :param X_test: Testing set features
    :param y_train: Training set labels
    :return: tuple of (best classifier instance, clf predictions, dict of best parameters, grid score)
    """
    # Parameter Grid - dictionary of parameters (map parameter names to values to be searched)
    param_grid = [
        {'C': [0.01, 0.1, 1, 10, 100, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10], 'kernel': ['linear']},
        {'C': [0.01, 0.1, 1, 10, 100, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10], 'kernel': ['rbf']},
        {'C':[0.01, 0.1, 1, 10, 100, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10], 'degree': [2], 'kernel': ['poly']}
    ]

    
    # Blank clf instance
    blank_clf = SVR()

    # Grid Search - Hyperparameters Optimization
    clf = grid_search.GridSearchCV(blank_clf, param_grid, n_jobs=-1)  # classifier + optimal parameters
    clf = clf.fit(X_train, y_train)  # fitted classifier
    best_est = clf.best_estimator_
    clf_pred = best_est.predict(X_test)

    best_params = clf.best_params_  # best parameters identified by grid search
    score = clf.best_score_  # best grid score
    return (best_est, clf_pred, best_params, score)

In [42]:
svr_model, svr_pred, svr_p, svr_score = gridsearch(scaled_X_train, scaled_X_test, y_train)
print "Best Parameters: ", svr_p
print "Best Grid Search Score: ", svr_score
print "Best Estimator: ", svr_model, "\n"

Best Parameters:  {'kernel': 'linear', 'C': 1000, 'gamma': 0.0001}
Best Grid Search Score:  0.0510005223002
Best Estimator:  SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.0001,
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False) 



Linear kernel was determined to be the most optimal for SVR model. Thus, the model is similar to Linear Regression.

#### Compare Predictions

In [43]:
svr_pred[-10:]

array([  2.53905234,   2.55147744,   6.33025238,   2.33192863,
         2.53241604,   2.83256744,  13.31739346,   2.91845799,
         1.55092954,   6.40806963])

In [44]:
y_test[-10:]

array([  4.6149,   6.8684,  13.8397,   0.3632,   1.6397,   3.0585,
         3.0521,   0.2826,   3.5426,   0.8828])

### Metrics

#### Root Mean Squared Error (RMSE)
Indicates the *absolute fit* of the model to the data. In other words, how close the observed data points are to the model's predicted values.

Also, it is the standard deviation of unexplained variance (residuals).

In [45]:
rmse = RMSE(svr_pred, y_test)
print "RMSE = %.3f days" % rmse

RMSE = 10.684 days


In [46]:
print "[%.2f,%.2f]" % (y_test.min(), y_test.max())

[0.00,71.01]


The SVR model has an RMSE value of 10.68 days. The value is somewhat small relative to the range of the `dependent variable` (average ICU LOS). Thus, indicating that model has decent accuracy in predicting the response.

#### Mean Absolute Error
The Mean Absolute Error measures how close the model's predictions are to the observed values.

In [47]:
mae = MAE(svr_pred, y_test)
print "MAE = %.3f days" % mae

MAE = 1.643 days


The average difference between prediction and observation is 1.64 days, which is low.

#### R<sup>2</sup>: Coefficient of Determination
R<sup>2</sup> quantifies the goodness of fit of the linear model. More specifically, it depicts the predictive power of the model.

Range: [0, 1]

In [48]:
r2(svr_pred, y_test)

0.074394243452860609

The R<sup>2</sup> value is approximately 0.074, which is low and indicates that the model does not have strong predictive power.